### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 7
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 7, 17), datetime.date(2023, 7, 16))

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-07-17
yesterday: 2023-07-14


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(30, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(186, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(30, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(30, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,BDMS,-24,XXX,SET50,1000,24.0,28.5,0.75,2.702703,0,2022-12-22
B,BEM,-18,52WL,SET50,2500,7.8,8.7,0.2,2.352941,0,2022-12-22
B,BH,-84,5pct,SET50,200,169.0,222.0,2.0,0.909091,0,2022-12-22
B,CPNREIT,-7,5pct,SET,5000,10.2,10.9,0.0,0.0,1,2023-05-23
B,DIF,-21,5pct,SET,3000,9.2,10.5,-0.2,-1.869159,0,2023-05-15
B,EA,-19,52WL,SET50,1200,52.0,56.75,0.5,0.888889,0,2022-12-22
B,GVREIT,-6,RD10pct,SET,10000,8.1,8.4,0.0,0.0,1,2023-05-26
B,JMT,-12,RD35pct,SET50,1200,33.0,36.0,0.5,1.408451,0,2023-04-11
B,LANNA,-13,,SET,1500,13.8,15.1,0.1,0.666667,0,2023-03-18
B,MCS,-8,RD60pct,SET,4000,6.0,6.4,0.05,0.787402,1,2023-03-17
B,NER,-11,RD40pct,SET,5000,4.4,4.62,0.06,1.315789,0,2023-04-20
B,ORI,-6,52WL,SET100,3000,10.0,10.6,0.2,1.923077,1,2023-05-08
B,PTTEP,-42,,SET50,200,134.5,155.5,-3.0,-1.892744,0,2023-02-14
B,SENA,-6,C4.48,SET,7000,2.88,3.0,0.0,0.0,1,2023-05-11
B,SINGER,-20,RD65pct,SET100,3000,8.3,9.3,0.2,2.197802,0,2023-05-03
B,STA

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
28,S,TFFIF,3,DOS,SET,10000,7.70,7.55,-0.05,-0.66,1,2023-05-31,77000.00
29,S,WHART,3,5pct,SET,10000,11.00,10.70,0.00,0.00,1,2023-05-23,110000.00


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
3,B,CPNREIT,-7,5pct,SET,5000,10.20,10.90,0.00,0.00,1,2023-05-23,51000.00
6,B,GVREIT,-6,RD10pct,SET,10000,8.10,8.40,0.00,0.00,1,2023-05-26,81000.00
9,B,MCS,-8,RD60pct,SET,4000,6.00,6.40,0.05,0.79,1,2023-03-17,24000.00
11,B,ORI,-6,52WL,SET100,3000,10.00,10.60,0.20,1.92,1,2023-05-08,30000.00
13,B,SENA,-6,C4.48,SET,7000,2.88,3.00,0.00,0.00,1,2023-05-11,20160.00
17,B,TMT,-7,RD30pct,SET,3000,6.95,7.30,0.00,0.00,1,2023-04-18,20850.00
20,S,BBL,7,DTD,SET50,400,163.50,160.00,2.00,1.27,1,2023-04-21,65400.00
28,S,TFFIF,3,DOS,SET,10000,7.70,7.55,-0.05,-0.66,1,2023-05-31,77000.00
29,S,WHART,3,5pct,SET,10000,11.00,10.70,0.00,0.00,1,2023-05-23,110000.00


In [17]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,BDMS,-24,XXX,SET50,1000,24.00,28.50,0.75,2.70,0,2022-12-22,24000.00
1,B,BEM,-18,52WL,SET50,2500,7.80,8.70,0.20,2.35,0,2022-12-22,19500.00
2,B,BH,-84,5pct,SET50,200,169.00,222.00,2.00,0.91,0,2022-12-22,33800.00
4,B,DIF,-21,5pct,SET,3000,9.20,10.50,-0.20,-1.87,0,2023-05-15,27600.00
5,B,EA,-19,52WL,SET50,1200,52.00,56.75,0.50,0.89,0,2022-12-22,62400.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,S,IVL,14,5pct,SET50,2400,37.75,34.25,-1.00,-2.84,0,2023-05-26,90600.00
24,S,JASIF,12,CUT,SET,10000,7.50,6.90,-0.05,-0.72,0,2023-05-19,75000.00
25,S,JMART,15,5pct,SET50,3600,17.90,16.40,0.00,0.00,0,2023-04-11,64440.00
26,S,PTG,18,DTD,SET100,3600,12.10,10.30,0.10,0.98,0,2022-12-22,43560.00


In [18]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [19]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [20]:
df['percent'] = df['%change']

In [21]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
23,S,IVL,14,5pct,SET50,2400,37.75,34.25,-1.00,-2.84,0,2023-05-26,90600.00,-2.84
27,S,PTTGC,57,RD45pct,SET50,600,53.25,39.00,-1.50,-3.70,0,2023-02-27,31950.00,-3.70


In [22]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
0,B,BDMS,-24,XXX,SET50,1000,24.00,28.50,0.75,2.70,0,2022-12-22,24000.00,2.70
1,B,BEM,-18,52WL,SET50,2500,7.80,8.70,0.20,2.35,0,2022-12-22,19500.00,2.35
14,B,SINGER,-20,RD65pct,SET100,3000,8.30,9.30,0.20,2.20,0,2023-05-03,24900.00,2.20
15,B,STA,-12,RD60pct,SET100,1500,16.20,17.40,0.40,2.35,0,2023-04-19,24300.00,2.35
16,B,SYNEX,-22,RND65pct,SET,2000,10.00,12.20,0.40,3.39,0,2023-03-09,20000.00,3.39


In [23]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
6,B,GVREIT,-6,RD10pct,SET,10000,8.10,8.40,0.00,0.00,1,2023-05-26,81000.00,0.00
13,B,SENA,-6,C4.48,SET,7000,2.88,3.00,0.00,0.00,1,2023-05-11,20160.00,0.00
11,B,ORI,-6,52WL,SET100,3000,10.00,10.60,0.20,1.92,1,2023-05-08,30000.00,1.92
17,B,TMT,-7,RD30pct,SET,3000,6.95,7.30,0.00,0.00,1,2023-04-18,20850.00,0.00
3,B,CPNREIT,-7,5pct,SET,5000,10.20,10.90,0.00,0.00,1,2023-05-23,51000.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,B,DIF,-21,5pct,SET,3000,9.20,10.50,-0.20,-1.87,0,2023-05-15,27600.00,-1.87
16,B,SYNEX,-22,RND65pct,SET,2000,10.00,12.20,0.40,3.39,0,2023-03-09,20000.00,3.39
0,B,BDMS,-24,XXX,SET50,1000,24.00,28.50,0.75,2.70,0,2022-12-22,24000.00,2.70
12,B,PTTEP,-42,NaN,SET50,200,134.50,155.50,-3.00,-1.89,0,2023-02-14,26900.00,-1.89


### Sell recently purchased stocks if profitable

In [24]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(108, 4)

In [25]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [26]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(105, 8)

In [27]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
4,BBL,2023-07-07,400,155.00,160.00,2000.00,1.27,3.23
95,TFFIF,2023-07-06,10000,7.55,7.55,0.00,-0.66,0.00
107,WHART,2022-05-17,10000,10.50,10.70,2000.00,0.00,1.90


### Profit per Day

In [28]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 7
ORDER BY sells.date DESC, name


In [29]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,KTC,2023,7,45.00,50.00,-5.00,900,"40,500.00","45,000.00","-4,500.00",-10.00,28
1,JMART,2023,7,17.90,17.00,0.90,"1,800","32,220.00","30,600.00","1,620.00",5.29,13


In [30]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [31]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,KTC,2023,7,45.00,50.00,-5.00,900,"40,500.00","45,000.00","-4,500.00",-10.00,28,-0.357143
1,JMART,2023,7,17.90,17.00,0.90,"1,800","32,220.00","30,600.00","1,620.00",5.29,13,0.407240


In [32]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.03%
